# GraphRAG do conteúdo dos pdfs das bulas de medicamentos salvos no Neo4j

In [ ]:
from src.connetion.embeddings import EmbeddingsModel
from src.connetion.chat_model import LLMModel
from src.connetion.graph_db import KgDatabaseConnetion
from src.agents.tools.graph_rag import GraphRAG

2025-06-15 20:32:59.247 | INFO     | src.config:<module>:12 - PROJ_ROOT path is: /home/marilia/git/pos/tcc_ufg_akcit_nlp


In [2]:
provider = "google"
temperature = 0
max_tokens = None

llm = LLMModel(
    provider=provider,
    temperature=temperature,
    max_tokens=max_tokens
).llm

In [3]:
provider = "hf"
embedding = EmbeddingsModel(
    provider=provider,
).embeddings

/home/marilia/git/pos/tcc_ufg_akcit_nlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
db = KgDatabaseConnetion(
    llm=llm,
    embedding=embedding,
)

In [5]:
db.vector.show_vector_constraints_and_indexes()

([{'id': 3,
   'name': 'constraint_907a464e',
   'type': 'UNIQUENESS',
   'entityType': 'NODE',
   'labelsOrTypes': ['__Entity__'],
   'properties': ['id'],
   'ownedIndex': 'constraint_907a464e',
   'propertyType': None}],
 [{'name': 'vector',
   'type': 'VECTOR',
   'labelsOrTypes': ['Document'],
   'properties': ['embedding'],
   'options': {'indexProvider': 'vector-2.0',
    'indexConfig': {'vector.hnsw.m': 16,
     'vector.hnsw.ef_construction': 100,
     'vector.dimensions': 768,
     'vector.similarity_function': 'COSINE',
     'vector.quantization.enabled': True}}}])

In [6]:
print(db.graph.graph.schema)

Node properties:
- **Medicamento**
  - `id`: STRING Available options: ['Medicamento']
- **Dose**
  - `id`: STRING Available options: ['Dose_Recomendada']
- **Sintoma**
  - `id`: STRING Available options: ['Sintomas']
- **Problema de saúde**
  - `id`: STRING Available options: ['Problemas_De_Saude']
- **Pessoa**
  - `id`: STRING Available options: ['Medico']
- **Document**
  - `id`: STRING Available options: ['a86bce566e661a48338e0e5dd2436b91']
  - `text`: STRING Available options: ['4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO']
- **Conceito**
  - `id`: STRING Available options: ['Doenca_Grave', 'Sintoma', 'Saude', 'Dose_Recomendada', 'Superdose']
- **Profissional**
  - `id`: STRING Available options: ['Medico']
- **Medicament**
  - `id`: STRING Available options: ['Medicamento']
- **Dosage**
  - `id`: STRING Available options: ['Dose_Recomendada']
- **Healthissue**
  - `id`: STRING Available options: ['Problemas_De_Saude', 'Doencas_Graves']
- **Person**
  - `id`: STRING Availab

In [7]:
print(db.graph.graph.get_schema)

Node properties:
- **Medicamento**
  - `id`: STRING Available options: ['Medicamento']
- **Dose**
  - `id`: STRING Available options: ['Dose_Recomendada']
- **Sintoma**
  - `id`: STRING Available options: ['Sintomas']
- **Problema de saúde**
  - `id`: STRING Available options: ['Problemas_De_Saude']
- **Pessoa**
  - `id`: STRING Available options: ['Medico']
- **Document**
  - `id`: STRING Available options: ['a86bce566e661a48338e0e5dd2436b91']
  - `text`: STRING Available options: ['4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO']
- **Conceito**
  - `id`: STRING Available options: ['Doenca_Grave', 'Sintoma', 'Saude', 'Dose_Recomendada', 'Superdose']
- **Profissional**
  - `id`: STRING Available options: ['Medico']
- **Medicament**
  - `id`: STRING Available options: ['Medicamento']
- **Dosage**
  - `id`: STRING Available options: ['Dose_Recomendada']
- **Healthissue**
  - `id`: STRING Available options: ['Problemas_De_Saude', 'Doencas_Graves']
- **Person**
  - `id`: STRING Availab

In [ ]:
grag = GraphRAG(llm=llm, db=db)

In [9]:
grag.retriever("O que devo saber antes de tomar o medicamento?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Document {id: 'a86bce566e661a48338e0e5dd2436b91'})-[r:MENTIONS]->(m:Medicamento)
WHERE d.text CONTAINS 'O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO'
RETURN m.id

Full Context:
[{'m.id': 'Medicamento'}]

> Finished chain.
** QA chain results **
{'query': 'O que devo saber antes de tomar o medicamento?', 'result': 'Não sei.', 'intermediate_steps': [{'query': "cypher\nMATCH (d:Document {id: 'a86bce566e661a48338e0e5dd2436b91'})-[r:MENTIONS]->(m:Medicamento)\nWHERE d.text CONTAINS 'O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO'\nRETURN m.id\n"}, {'context': [{'m.id': 'Medicamento'}]}]}
[{'m.id': 'Medicamento'}]
** Vector similarity results **
[Document(metadata={}, page_content='4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO?  \nVocê não deve tomar mais do que a dose recomendada (superdose) para provocar maior alívio, pois pode causar sérios problemas de saúde. Você deve consultar seu médico se a dor ou

'Antes de usar este medicamento, você não deve tomar mais do que a dose recomendada para evitar sérios problemas de saúde.  Se a dor ou febre persistirem, piorarem ou surgirem novos sintomas, você deve consultar seu médico, pois estes podem ser sinais de doenças graves.'

In [10]:
grag.retriever(question="What was the cast of the Casino?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie) WHERE m.title = "Casino" RETURN p.name

Full Context:
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]

> Finished chain.
** QA chain results **
{'query': 'What was the cast of the Casino?', 'result': 'The cast of Casino included Robert De Niro, Joe Pesci, Sharon Stone, and James Woods.', 'intermediate_steps': [{'query': 'cypher\nMATCH (p:Person)-[:ACTED_IN]->(m:Movie) WHERE m.title = "Casino" RETURN p.name\n'}, {'context': [{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]}]}
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]
** Vector similarity results **
[Document(metadata={}, page_content='4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO?  \nVocê não deve tomar mais do que a dose recomendada 

'Based on the provided data, the cast of Casino included Robert De Niro, Joe Pesci, Sharon Stone, and James Woods.'

In [11]:
grag.retriever(question="Me fale sobre o covid")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Document)-[:MENTIONS]->(c:Conceito) WHERE d.id = 'a86bce566e661a48338e0e5dd2436b91' AND c.id CONTAINS 'covid' RETURN c
Full Context:
[]

> Finished chain.
** QA chain results **
{'query': 'Me fale sobre o covid', 'result': 'Não tenho informações sobre o covid.', 'intermediate_steps': [{'query': "MATCH (d:Document)-[:MENTIONS]->(c:Conceito) WHERE d.id = 'a86bce566e661a48338e0e5dd2436b91' AND c.id CONTAINS 'covid' RETURN c"}, {'context': []}]}
[]
** Vector similarity results **
[Document(metadata={}, page_content='4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO?  \nVocê não deve tomar mais do que a dose recomendada (superdose) para provocar maior alívio, pois pode causar sérios problemas de saúde. Você deve consultar seu médico se a dor ou febre continuar ou piorar, ou se surgirem novos sintomas, pois estes sintomas podem ser sinais de doenças graves.')]


 Context:
 
        # Structured data:
        ['Não sei r

"I'm sorry, but the provided text does not contain information about COVID-19.  The text only discusses exceeding the recommended dosage of a medication and the importance of consulting a doctor if symptoms worsen or new symptoms appear."

In [12]:
grag.retriever(question="Você sabe algo sobre LLMs?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Document {id: 'a86bce566e661a48338e0e5dd2436b91'})-[r:MENTIONS]->(n) RETURN d,r,n

Full Context:
[{'d': {'id': 'a86bce566e661a48338e0e5dd2436b91', 'text': '4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO?  \nVocê não deve tomar mais do que a dose recomendada (superdose) para provocar maior alívio, pois pode causar sérios problemas de saúde. Você deve consultar seu médico se a dor ou febre continuar ou piorar, ou se surgirem novos sintomas, pois estes sintomas podem ser sinais de doenças graves.', 'embedding': [0.05720214918255806, -0.03908204659819603, -0.006184726487845182, -0.0030857762321829796, 0.022133415564894676, 0.011207369156181812, -0.008866087533533573, 0.028748128563165665, 0.028442904353141785, -0.0059473454020917416, -0.0016515799798071384, 0.0034806428011506796, 0.02001987397670746, 0.028822530061006546, -0.03205113112926483, -0.04275377839803696, -0.022841861471533775, -0.00986627116799354

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250000
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 32
}
].


'Não tenho informações sobre LLMs neste contexto.  Meu conhecimento se limita às informações fornecidas na bula de medicamento.'